In [12]:
from Utils.univariate_featureSelection import featureSelectionResults, get_selectedFeatures
from Utils.setupDataset import get_dataset
from Utils.feature_vector import get_feature_names
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import f1_score, make_scorer
from sklearn import metrics as skmetrics
from Utils.selection_metrics import SelectionMetrics
import pandas as pd
from Utils.lasso_selections import get_LassoSelectionResults
from Utils.univariate_featureSelection import get_metrics, get_selectedFeatures

from sklearn import preprocessing

fnames = get_feature_names()

m_dev = get_dataset("Datasets/M_DEV_dataset.pickle")
w_dev = get_dataset("Datasets/W_DEV_dataset.pickle")
m_val = get_dataset("Datasets/M_VAL_dataset.pickle")
w_val = get_dataset("Datasets/W_VAL_dataset.pickle")

m_filename = 'Datasets/M_SELECTIONS_univariate.csv'
w_filename = 'Datasets/W_SELECTIONS_univariate.csv'


m_rfe = 'Datasets/M_SELECTIONS_RFE.csv'
w_rfe = 'Datasets/W_SELECTIONS_RFE.csv'

m_sel_lasso = "Datasets/M_SELECTIONS_lasso.pickle"
w_sel_lasso = "Datasets/W_SELECTIONS_lasso.pickle"


# 1. load data (DEV set)
X_m = m_dev["data"]
y_m = m_dev["target"]
X_m_val = m_val["data"]
y_m_val = m_val["target"]

X_w = w_dev["data"]
y_w = w_dev["target"]
X_w_val = w_val["data"]
y_w_val = w_val["target"]

# apply feature selection


# 2. scale data (TODO)

def test_SVM(train, test, kernel='linear', gamma=1.0, C=10.0, scale=False):
    X = train["data"]
    y = train["target"]
    X_test = test["data"]
    y_test = test["target"]
    
    if scale:
        scaler = preprocessing.StandardScaler().fit(X)
        X = scaler.transform(X)
        X_test = scaler.transform(X_test)
    
    clf = None
    if kernel == 'linear':
        clf = SVC(kernel=kernel, C=C)
        print 'linear!'
    else:
        clf = SVC(kernel='rbf', C=C, gamma=gamma)
    
    
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    confusion = skmetrics.confusion_matrix(y_test, y_pred)
    TP = confusion[1,1]
    TN = confusion[0,0]
    FP = confusion[0,1]
    FN = confusion[1,0]
    accuracy = skmetrics.accuracy_score(y_test, y_pred)
    recall = skmetrics.recall_score(y_test,y_pred)

    precision = skmetrics.precision_score(y_test, y_pred)
    f1 = 2*precision*recall/(precision+recall)
    metrics = np.array([ f1, precision, recall, accuracy, TN, FP, FN, TP])
    return metrics


In [7]:
m_m = test_SVM(m_dev, m_val, kernel='rbf', gamma=0.0001, C= 100)
m_m

array([   0.78125   ,    0.78125   ,    0.78125   ,    0.78294574,
        102.        ,   28.        ,   28.        ,  100.        ])

In [9]:
m2m = test_SVM(m_dev, m_val, kernel='rbf', gamma=0.00001, C= 1000)
m2m

array([   0.79389313,    0.7761194 ,    0.8125    ,    0.79069767,
        100.        ,   30.        ,   24.        ,  104.        ])

In [10]:
m_lin = test_SVM(m_dev, m_val, kernel='linear',  C= 10)
m_lin

array([  0.75294118,   0.75590551,   0.75      ,   0.75581395,
        99.        ,  31.        ,  32.        ,  96.        ])

In [11]:
m_lin2 = test_SVM(m_dev, m_val, kernel='linear',  C= 1000)
m_lin2

array([   0.75697211,    0.77235772,    0.7421875 ,    0.76356589,
        102.        ,   28.        ,   33.        ,   95.        ])

In [13]:
m_lin2 = test_SVM(m_dev, m_val, kernel='linear',  C= 200)
m_lin2

linear!


array([   0.7755102 ,    0.81196581,    0.7421875 ,    0.78682171,
        108.        ,   22.        ,   33.        ,   95.        ])

In [16]:
m2m = test_SVM(m_dev, m_val, kernel='rbf', gamma=0.00005, C= 1000)
m2m

array([   0.744     ,    0.76229508,    0.7265625 ,    0.75193798,
        101.        ,   29.        ,   35.        ,   93.        ])

In [49]:
m2m = test_SVM(m_dev, m_val, kernel='rbf', gamma=0.000075, C= 150) # gamma=0.00001, C= 1000
m2m

array([   0.7890625 ,    0.7890625 ,    0.7890625 ,    0.79069767,
        103.        ,   27.        ,   27.        ,  101.        ])

In [66]:
m2m = test_SVM(m_dev, m_val, kernel='rbf', gamma=0.00005, C= 200)
m2m

array([   0.79844961,    0.79230769,    0.8046875 ,    0.79844961,
        103.        ,   27.        ,   25.        ,  103.        ])

In [87]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 100, scale=True)
w

linear!


array([   0.87325652,    0.93023256,    0.82285714,    0.88029782,
        817.        ,   54.        ,  155.        ,  720.        ])

In [88]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 100)
w

linear!


array([   0.87307236,    0.90752158,    0.84114286,    0.87743414,
        796.        ,   75.        ,  139.        ,  736.        ])

In [89]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 200, scale=True)
w

linear!


array([   0.87014563,    0.92755498,    0.81942857,    0.87743414,
        815.        ,   56.        ,  158.        ,  717.        ])

In [90]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 220, scale=True)
w

linear!


array([   0.87151515,    0.92774194,    0.82171429,    0.87857961,
        815.        ,   56.        ,  156.        ,  719.        ])

In [91]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 300, scale=True)
w

linear!


array([   0.8716707 ,    0.92664093,    0.82285714,    0.87857961,
        814.        ,   57.        ,  155.        ,  720.        ])

In [92]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 300)
w

linear!


array([   0.87287639,    0.89543269,    0.85142857,    0.87571592,
        784.        ,   87.        ,  130.        ,  745.        ])

In [93]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 220)
w

linear!


array([   0.87279152,    0.90036452,    0.84685714,    0.87628866,
        789.        ,   82.        ,  134.        ,  741.        ])

In [94]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 200)
w

linear!


array([   0.86725664,    0.89634146,    0.84      ,    0.87113402,
        786.        ,   85.        ,  140.        ,  735.        ])

In [96]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 10, scale=True)
w

linear!


array([   0.87310261,    0.93134715,    0.82171429,    0.88029782,
        818.        ,   53.        ,  156.        ,  719.        ])

In [99]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 50, scale=True)
w

linear!


array([   0.87378641,    0.93143596,    0.82285714,    0.88087056,
        818.        ,   53.        ,  155.        ,  720.        ])

In [100]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 70, scale=True)
w

linear!


array([   0.87257282,    0.9301423 ,    0.82171429,    0.87972509,
        817.        ,   54.        ,  156.        ,  719.        ])

In [101]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 20, scale=True)
w

linear!


array([   0.87241798,    0.93125811,    0.82057143,    0.87972509,
        818.        ,   53.        ,  157.        ,  718.        ])

In [102]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 20)
w

In [103]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 50)
w

array([   0.8697204 ,    0.90694789,    0.83542857,    0.87457045,
        796.        ,   75.        ,  144.        ,  731.        ])

In [104]:
w = test_SVM(w_dev, w_val, kernel='linear', C= 70)
w

linear!


array([   0.87179487,    0.91147132,    0.83542857,    0.8768614 ,
        800.        ,   71.        ,  144.        ,  731.        ])